# Logic gates using neural networks via Keras (Tensorflow)
https://machinelearningmastery.com/tutorial-first-neural-network-python-keras/

Here we are using Keras which is supposedly a high level API for Tensorflow, or that it uses Tensorflow as backend.

However, it has been recently advocated that one should use 'tf.keras' instead. This is essentially, keras being included as a submodule inside tensorflow itself.

https://www.pyimagesearch.com/2019/10/21/keras-vs-tf-keras-whats-the-difference-in-tensorflow-2-0/

In [202]:
import numpy as np
# import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras import optimizers

## Inputs

In [203]:
inputs = np.array([[0.,0.,1.,1.],[0.,1.,0.,1.]]).T
print(inputs)

[[0. 0.]
 [0. 1.]
 [1. 0.]
 [1. 1.]]


## Expected outputs


In [204]:
# AND function
outputAND = np.array([0.,0.,0.,1.])
outputAND = np.asarray([outputAND]).T
# OR function
outputOR = np.array([0.,1.,1.,1.])
outputOR = np.asarray([outputOR]).T
# NAND function
outputNAND = np.array([1.,1.,1.,0.])
outputNAND = np.asarray([outputNAND]).T
# XOR function
outputXOR = np.array([0.,1.,1.,1.])
outputXOR = np.asarray([outputXOR]).T

## Set initial weights and biases

In [205]:
# Initial guesses for weights
w1 = 0.30
w2 = 0.55
w3 = 0.20
w4 = 0.45
w5 = 0.50
w6 = 0.35
w7 = 0.15
w8 = 0.40
w9 = 0.25

# Initial guesses for biases
b1 = 0.60
b2 = 0.05

# need to use a list instead of a numpy array, since the 
#weight matrices at each layer are not of the same dimensions
weights = [] 
# Weights for layer 1 --> 2
weights.append(np.array([[w1,w4],[w2, w5], [w3, w6]]))
# Weights for layer 2 --> 3
weights.append(np.array([[w7, w8, w9]]))
# List of biases at each layer
biases = []
biases.append(np.array([b1,b1,b1]))
biases.append(np.array([b2]))

weightsOriginal = weights
biasesOriginal = biases

print('Weights matrices: ',weights)
print('Biases: ',biases)





Weights matrices:  [array([[0.3 , 0.45],
       [0.55, 0.5 ],
       [0.2 , 0.35]]), array([[0.15, 0.4 , 0.25]])]
Biases:  [array([0.6, 0.6, 0.6]), array([0.05])]


## Some more settings

In [206]:
nLayers = 2
nSamples = 4
eeta = 0.5

## Define model

In [207]:
# define the keras model
model = Sequential()
model.add(Dense(3, input_dim=2, activation='sigmoid', use_bias=True))
model.add(Dense(1, activation='sigmoid', use_bias=True))

## Model summary

In [208]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_2 (Dense)              (None, 3)                 9         
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 4         
Total params: 13
Trainable params: 13
Non-trainable params: 0
_________________________________________________________________


## Check the initial weights and biases for each layer

Note how the weights matrix is not 3x2 but rather 2x3

In [209]:
for i in range(nLayers):
    print('\n Weights for layer ',i+1)
    print(model.layers[i].get_weights()[0])
    print('\n Biases for layer ',i+1)
    print(model.layers[i].get_weights()[1])
# model.layers[0].get_biases()


 Weights for layer  1
[[-1.0776986   0.29754436  0.24226391]
 [ 0.15691936  0.22695196  0.70656633]]

 Biases for layer  1
[0. 0. 0.]

 Weights for layer  2
[[ 0.18757391]
 [-0.49709475]
 [ 0.16926765]]

 Biases for layer  2
[0.]


## Change initial weights and biases for each layer

In [210]:
# Layer 1
model.layers[0].set_weights([weightsOriginal[0].T, biasesOriginal[0]])

# Layer 2
model.layers[1].set_weights([weightsOriginal[1].T, biasesOriginal[1]])

## Compile model

In [211]:
# compile the keras model

# In the following manner we can't set the learning rate of the optimizer
# model.compile(loss='mse', optimizer='sgd', metrics=['mse'])

# So use the following instead
model.compile(loss='mse', optimizer=optimizers.SGD(learning_rate=0.5), metrics=['mse'])



## Forward feed

In [212]:
model.evaluate(inputs, outputAND, batch_size=4)

4/1 [========================================================================================================================] - 0s 36ms/sample - loss: 0.3434 - mse: 0.3434


[0.3434138894081116, 0.3434139]

## Does tf.model.evaluate change the weights and biases?

In [213]:
for i in range(nLayers):
    print('\n Weights for layer ',i+1)
    print(model.layers[i].get_weights()[0])
    print('\n Biases for layer ',i+1)
    print(model.layers[i].get_weights()[1])
# model.layers[0].get_biases()


 Weights for layer  1
[[0.3  0.55 0.2 ]
 [0.45 0.5  0.35]]

 Biases for layer  1
[0.6 0.6 0.6]

 Weights for layer  2
[[0.15]
 [0.4 ]
 [0.25]]

 Biases for layer  2
[0.05]


From the above, we can be sure that it does not change the weights and biases

## Fit 1 epoch  (forward feed, backpropagation, updating the weights, biases)

Let us just try to see and compare the error after just 1 epoch. 

We should expect the model to perform forward feed, calculate loss/error,
perform backpropagation,
and adjust the weights and biases based on the learning rate.

In [214]:
# fit the keras model on the dataset
model.fit(inputs, outputAND, epochs=1, batch_size=4)

Train on 4 samples
4/4 [==============================] - 0s 93ms/sample - loss: 0.3434 - mse: 0.3434


## Now the weights and biases must have been updated

In [215]:
for i in range(nLayers):
    print('\n Weights for layer ',i+1)
    print(model.layers[i].get_weights()[0])
    print('\n Biases for layer ',i+1)
    print(model.layers[i].get_weights()[1])
# model.layers[0].get_biases()


 Weights for layer  1
[[0.29931322 0.54829705 0.1988662 ]
 [0.4493876  0.49822634 0.34898397]]

 Biases for layer  1
[0.5969832 0.5921526 0.5948988]

 Weights for layer  2
[[0.08719039]
 [0.33587077]
 [0.1880536 ]]

 Biases for layer  2
[-0.04234041]


## Now let us do a forward feed again and calculate the loss/error

In [216]:
out = model.evaluate(inputs, outputAND, batch_size=4)
print(out)

4/1 [========================================================================================================================] - 0s 738us/sample - loss: 0.3041 - mse: 0.3041
[0.30411168932914734, 0.3041117]


### The above result, compares well with the PyTorch result as well as the result from my own implementation (when biases are updated independently)

## Now let us let the model train for 10^4 epochs

In [217]:
%%time
# fit the keras model on the dataset
model.fit(inputs, outputAND, epochs=10**4, batch_size=4, verbose=0)

CPU times: user 28.2 s, sys: 4.43 s, total: 32.6 s
Wall time: 16.3 s


Turns out that this was quite slow. At first, I thought that the problem was that it was printing at each epoch. So I set the verbose=0 (silent). But still, it was incredibly slow compared to my implementation as well as PyTorch, even though we used own for loop in both the cases.

## Well, now let us check the output as well as the new weights and biases

In [218]:
out = model.evaluate(inputs, outputAND, batch_size=4, verbose=1)
print(out)

4/1 [========================================================================================================================] - 0s 752us/sample - loss: 2.3155e-04 - mse: 2.3155e-04
[0.00023155275266617537, 0.00023155275]


In [219]:

for i in range(nLayers):
    print('\n Weights for layer ',i+1)
    print(model.layers[i].get_weights()[0])
    print('\n Biases for layer ',i+1)
    print(model.layers[i].get_weights()[1])
# model.layers[0].get_biases()


 Weights for layer  1
[[-0.06779443  3.6265924  -1.9713068 ]
 [ 0.05656041  3.656049   -1.9379458 ]]

 Biases for layer  1
[ 0.74201846 -5.253187    2.5809646 ]

 Weights for layer  2
[[-0.30047134]
 [ 8.487755  ]
 [-4.2936563 ]]

 Biases for layer  2
[-2.5681338]


## Now also let us have a look at the predictions for the sake of the tutorial

In [220]:
# make probability predictions with the model
predictions = model.predict(inputs)
print(predictions)

[[0.00120634]
 [0.01536182]
 [0.01545322]
 [0.97878754]]
